# Tutorial 2: Estimating linear diffusion ages

This tutorial uses outputs generated by Tutorial 1. Specifically, we will make use of the saved Riser instance files. If you haven't completed Tutorial 1, do it before you run this one.

In [ ]:
import riserfit as rf
import os
import matplotlib.pyplot as plt
import numpy as np

### Linear diffusion ages
riserfit uses scipy's optimize functions to find the best combination of parameters (riser height, far-field slope, diffusion age, initial slope, and offsets in d and z directions) that result in a close match between data and model. 

The only pre-processing step is that we need to ensure that profiles are at least approximately centered at the coordinate origin. This is because diffusion creates profiles that have rotational symmetry about the coordinate origin. If you take a look at the plot in the first tutorial, you will see that the profiles are already centered! This step is performed automatically by `rf.construct_z_profiles_from_centerpoints`.
If you have loaded pandas dataframes that contain individual riser profiles, you can center them using `rf.center_elevation_profiles()`. This might be necessary if you have recorded riser profiles with a GNSS device. Luckily, we don't need to care about that.

Once our profiles are centered, we can run `rf.Risers.compute_best_linear_diffusion_fit()`, which does most of the processing automatically. Among many other things, this will generate precise offsets that we can use to properly center our profiles. This needs to be done before running the next processing steps!

In [ ]:
# set working directory and parameters
os.chdir(r"C:\\Users\\Lennart\\lennartGit\\personal\\riserfit\\Tutorials")

# load the two riser instances
terraces = ["T7", "T3"]
fnames = [f"\\Data\\Risers\\Instances\\{t}_Riser_instance.gz" for t in terraces]
risersT7 = rf.load_instance(fnames[0])
risersT3 = rf.load_instance(fnames[1])

# perform the morphological dating for T7 ...
risersT7.compute_best_linear_diffusion_fit(
    kt_range=(0, 100), # range of allowed kt values
    b_range=(-0.02, 0.02), # range of allowed far-field slope values
    theta_range=(0.2, 0.6), # range of allowed initial slope values
    d_off_range=(-10, 10), # allowed offset range
    z_off_range=(-1, 1), # allowed offset_range
    verbose=True # print the results to console?
); # semicolon because this method returns the instance itself and we do not need to see that.
risersT7.apply_d_z_offsets();

# ... and for T3
risersT3.compute_best_linear_diffusion_fit(
    kt_range=(0, 100), # range of allowed kt values
    b_range=(-0.02, 0.02), # range of allowed far-field slope values
    theta_range=(0.2, 0.6), # range of allowed initial slope values
    d_off_range=(-10, 10), # allowed offset range
    z_off_range=(-1, 1), # allowed offset_range
    verbose=False # do not show the results
); 
risersT3.apply_d_z_offsets();

We can also calculate uncertainties for our diffusion age estimates. Altough there is currently no way to calculate uncertainties that would satisfy a statistician (see [Avouac 1993](https://doi.org/10.1029/92JB01962)), an established method in morphological dating is to calculate a $\sigma$ value and identify the range of $kt$ that result in misfits (RMSE) that satisfy $\text{RMSE}(kt) \leq \text{RMSE}_\text{min} + 1\sigma$. Various methods to determine $\sigma$ have been proposed, but riserfit uses the most recent suggestions by [Wei et al. 2015](https://doi.org/10.1016/j.jseaes.2015.02.016) and [Xu et al. 2021](https://doi.org/10.1002/esp.5022): Assuming that the elevation residuals between modelled and measured profile are Gaussian, their sum will be a Chi$^2$ distribution with variance $\sigma$. 
$$ \sigma = \sqrt{\frac{2s^4}{n}}$$
where $s$ is the standard deviation of the residuals
$$ s^2 = \frac{1}{n-1}\sum_{i=1}^n(z_\text{model},i-z_\text{data},i)^2$$
Bounds on $kt$ are then identified by evaluating the inequality
$$ \text{RMSE}^2(kt) \leq \text{RMSE}^2_\text{min} + \sigma $$
(because $\sigma$ is now the variance). In riserfit, the simple wrapper for calculating uncertainties is `rf.Riser.calculate_kt_uncertainty()`

In [ ]:
risersT7.calculate_kt_uncertainty(
    dt = 0.01, # resolution of the approximate solution 
    max_iteration=1e4 # many iterations will ensure that the upper bound is found.
);

risersT3.calculate_kt_uncertainty(
    dt = 0.01, # resolution of the approximate solution 
    max_iteration=1e4, # many iterations will ensure that the upper bound is found.
    verbose=False
);

In [ ]:
# Let's compare the diffusion ages of T7 and T3.

bins = np.linspace(0, 10, 10)
fg, ax = plt.subplots(2, 1)
ax[0].hist(
    risersT7.best_kt,
    bins=bins
);
ax[0].set_title("T7") # this is the young terrace!

ax[1].hist(
    risersT3.best_kt,
    bins=bins
);
ax[1].set_title("T3") # this one is older!
ax[1].set_xlabel("Diffusion age kt [m^2]");

We should save the progress we've achieved! As explained in Tutorial 1, this can be done by saving the riser instances directly, or by exporting the data to a pandas dataframe.

In [ ]:
risersT7.save_instance(r"\\Data\\Risers\\Instances\\");
risersT3.save_instance(r"\\Data\\Risers\\Instances\\");

# let's look at the pandas dataframe
df = risersT7.build_Riser_instance_dataframe()
print(df.head())